# Tensorflow_MNIST_BasicNN

**References**
+ 텐서플로우코리아 : https://tensorflowkorea.gitbooks.io/tensorflow-kr/content/g3doc/tutorials/mnist/beginners/
+ 조대협의 블로그 : http://bcho.tistory.com/1154?category=555440

0~9의 손글씨 데이터인 MNIST를 사용해 숫자를 분류하는 가장 기본적인 신경망을 구현할 것이다. MNIST는 전처리가 필요없는 정제된 데이터셋으로 실습하기 가장 쉽고 간단하다. 총 60000장의 이미지로 학습용 55000장, 테스트용 10000장, 검증용 5000장으로 이미 나눠져 있다.

In [1]:
import tensorflow as tf
import numpy as np

## 1. 데이터 불러오기

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## 2. 텐서와 그래프 정의

한 층의 레이어를 사용해 가장 기본적인 신경망을 구현할 것이다.

1. **x** : 입력데이터가 들어가는 플레이스홀더이며 MNIST 이미지를 순차적으로 받아 흘려주는 역할을 한다. 따라서 tf.placeholder를 이용해 틀을 잡아주었다. x의 **None은 28x28 데이터가 임의 갯수 만큼 들어올** 수 있음을 의미하며, **784는 기존 28x28의 2차원 이미지를 1차원으로 펼쳐 받아올 것**임을 의미한다.

2. **W, b** : 학습을 통해 갱신해나갈 가중치와 편향치이므로 tf.Variable을 이용해 선언했다. 우리는 이미지를 10개의 Feature Map으로 추출한 뒤 소프트맥스를 통해 0~9 중 어느 클래스에 속하는지 예측하는 모델을 구현할 것이므로 **W와 b의 최종 차원을 10으로 설정**했다.

3. **y** : 예측값으로 Wx + b에 소프트맥스를 적용한 값이며 이를 텐서플로우로 표현한 코드는 아래와 같다.

In [7]:
# 텐서
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

# 그래프
y = tf.nn.softmax(tf.matmul(x, W) + b)

## 3. 손실함수 및 학습방법 설정

1. **y_** : Labeling된 실제 결과값으로 우리의 예측값 y와 비교해 Loss Function을 계산하기 위해 사용된다.
2. **cross_entropy** : Loss Function을 정의했다.
3. **train_step** : Learning Rate 0.5의 경사하강법을 적용해 손실함수를 최소화시키는 가중치와 편향치를 갱신해낸 최종 그래프이다. 이것을 세션에 넣어 돌리기만 하면 학습이 진행된다.

In [8]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

## 4. 학습 및 평가

### (1) 학습

+ with 절을 사용해 세션을 정의하고 모든 변수들을 초기화 한다.
+ 1000번의 반복학습을 시킬 건데, 모든 데이터를 사용하는 것이 아닌 100개의 랜덤 학습데이터만 뽑아 부분학습 시킨다. 이렇게 하면 연산량은 줄어들지만 1000번의 루프동안 모든 데이터를 사용해 학습한 효과를 낼 수 있으므로 효율적이다.

### (2) 평가

+ 예측값(y)과 실제값(y_)이 같은 경우 1을, 다를 경우 0을 반환하는 코드를 작성한다.
+ 텐서플로우 변수로 캐스팅하고 평균을 계산하면 맞은 비율, 즉 정확도가 나온다.

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 학습
    for i in range(1000):
        batch_x, batch_y = mnist.train.next_batch(100)
        sess.run(train_step, feed_dict={x : batch_x, y_ : batch_y})

    # 평가
    correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9159


+ **다음 노트북에서는 간단한 CNN 모델을 구현하여 기존 약 91.5%의 정확도를 99%까지 향상시켜 볼 것이다. (끝)**